In [1]:
import numpy as np
import os
import glob
from astropy.io import fits
from astropy.table import Table, unique, Column, vstack
import matplotlib.pyplot as plt
import pandas as pd
from qn_analysis.desi_utils import *

### Merge the separate zbest files from redrock into a single file.

In [2]:
#rrdir = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/runs/rr_v5_7_0/output/'
rrdir = '/global/cfs/projectdirs/desi/users/jfarr/MiniSV_data/SV0/rr_output/single_exp/'
f_out = '../rr_results/rr_SV0_00055628.fits'

fi = glob.glob(rrdir+'/zbest*00055628.fits')
print(len(fi))

1


In [3]:
## Concatenate output from all zbest files.
rr = fits.open(fi[0])
expid = int(fi[0].split('-')[-1][:-5])
rr_data = Table(rr[1].data)
expids = expid*np.ones(len(rr_data))
rr_data.add_column(Column(data=expids,name='EXPID',dtype='i8'))
rr.close()
if len(fi)>1:
    for i,f in enumerate(fi[1:]):
        rr = fits.open(f)
        expid = int(f.split('-')[-1][:-5])
        f_rr_data = Table(rr[1].data)
        expids = expid*np.ones(len(f_rr_data))
        f_rr_data.add_column(Column(data=expids,name='EXPID',dtype='i8'))
        rr_data = vstack([rr_data,f_rr_data])
        rr.close()
        print('Stacking zbest files: {:.2%}'.format((i+2)/len(fi)),end='\r')
print('')
print(len(rr_data))
rr_data.sort(['TARGETID','EXPID'])
rr_data[:5]


5000


TARGETID,CHI2,COEFF [10],Z,ZERR,ZWARN,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,NUMEXP,NUMTILE,EXPID
int64,float64,float64,float64,float64,int64,int64,str6,str20,int64,float64,int32,int32,int64
35185923930984962,8956.432122126222,144.92108740984966 .. -9.764885916264669,0.7168644027574248,8.216929118396619e-05,0,7688,GALAXY,,10,126.91072070598602,0,0,55628
35185923930985294,9052.539854824543,54.80832686925778 .. -0.646264386389226,0.8046316096688969,6.853589184390589e-05,0,7690,GALAXY,,10,19.694716170430183,0,0,55628
35185923930985431,7788.208816225913,2705.306301919515 .. 0.0,0.00014700201137964636,9.837811408052758e-06,0,7647,STAR,G,5,3050.8271644159286,0,0,55628
35185923930986052,8.999999999999996e+99,0.0 .. 0.0,-0.0019957014509158224,4.131159871613317e-48,1570,0,STAR,CV,3,1.942668892225729e+84,0,0,55628
35185923935177997,8981.338824473321,170.41585648657087 .. -1.0721569538804046,0.6802059076786762,0.00011287811223650476,0,7683,GALAXY,,10,322.1156157553196,0,0,55628


In [9]:
rr = fits.open(fi[0])
rr[1].data.dtype

dtype((numpy.record, [('TARGETID', '>i8'), ('CHI2', '>f8'), ('COEFF', '>f8', (10,)), ('Z', '>f8'), ('ZERR', '>f8'), ('ZWARN', '>i8'), ('NPIXELS', '>i8'), ('SPECTYPE', 'S6'), ('SUBTYPE', 'S20'), ('NCOEFF', '>i8'), ('DELTACHI2', '>f8'), ('NUMEXP', '>i4'), ('NUMTILE', '>i4')]))

In [101]:
rr_data.write(f_out,overwrite=True)

#### Open the qnAll file(s) we want, stitch them together and save.

In [74]:
fi_qn_loc = '/global/homes/j/jfarr/Projects/QuasarNET/cats/MiniSV/SV0/'
fi_qn_form = 'qnAll-*00055627_MINI_SV_QSO.fits'
fi_out = '../qn_results/qn_SV0_00055627_MINI_SV_QSO.fits'

In [75]:
import glob
fi_qn = glob.glob(fi_qn_loc+fi_qn_form)
data = []
for f_qn in fi_qn:
    qn = fits.open(f_qn)
    w = (~(qn[1].data['IN_TRAIN'].astype('bool')))
    f_data = np.array(qn[1].data[w])
    data.append(f_data)
data = np.concatenate(data)
table = Table(data)
table.sort('THING_ID')
table[:5]

THING_ID,PLATE,MJD,FIBERID,ZBEST,IS_QSO,IN_TRAIN,C_LINES [6],Z_LINES [6],C_LINES_BAL,Z_LINES_BAL
int64,bytes8,int32,int32,float64,int64,int64,float64,float64,float64,float64
35185923935178324,2014p067,3668,508,2.001752140353664,1,0,1.0 .. 0.0,2.001752140353664 .. -0.42869098027990227,0.9999992847442627,2.0116061584663516
35185923939370065,2017p067,1105,478,1.0805227620821087,0,0,1.519918441772461e-06 .. 0.0,2.791097577899391 .. -0.45295369733383306,0.0,1.4726900105505205
35185923939371431,2017p067,2471,397,2.1441034154176486,1,0,1.0 .. 0.0,2.1441034154176486 .. -0.44685963917036486,0.0,1.4704380911399126
35185923939373842,2017p067,4882,394,1.8791145626061105,0,0,0.0 .. 0.0,1.9989246811295676 .. -0.4486432478247404,1.8775463104248047e-06,2.553609239317933
35185923943563502,2019p067,238,470,3.4650784932566268,0,0,1.9669532775878906e-06 .. 0.0,3.4650784932566268 .. -0.44993861660440415,0.0,1.4015042417965864


In [76]:
table_unique = unique(table, keys=['PLATE', 'MJD', 'FIBERID'])
table = fits.BinTableHDU(table_unique)
prihdr = fits.Header()
prihdu = fits.PrimaryHDU(header=prihdr)
hdulist = fits.HDUList([prihdu, table])
hdulist.writeto(fi_out,overwrite=True)
hdulist.close()

### Convert a VI results csv file to a QN truth file.

In [28]:
f_vi = '/global/cfs/cdirs/desi/sv/vi/TruthTables/truth_table_QSO_v1.2.csv'
vi = pd.read_csv(f_vi, header=0)
vi[:10]

,TARGETID,Redrock z,best z,best quality,Redrock spectype,best spectype,all VI issues,all VI comments,merger comment,N_VI,...,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z
0,35185929937227607,1.4995,1.49950,1.0,GALAXY,GALAXY,--,Poor noise subtraction Maybe OII correct but...,None,5,...,696,0.728820,0.730633,1.221301,0.566604,0.568014,0.949472,0.916734,0.943119,0.967773
1,35185929937228335,0.4102,0.41020,3.6,GALAXY,GALAXY,--,--,none,5,...,657,0.228894,0.745470,1.842765,0.178230,0.580468,1.434888,0.916849,0.943198,0.967819
2,35185929937229340,1.4944,1.49440,4.0,QSO,QSO,--,--,none,5,...,652,2.102620,4.026039,6.276346,1.635143,3.130926,4.880920,0.917295,0.943508,0.967997
3,35185929941418536,1.0804,1.08032,3.8,GALAXY,GALAXY,--,Both galaxy and QSO in spectrum,none,5,...,194,1.567732,2.329047,4.268086,1.221243,1.814298,3.324784,0.917957,0.943966,0.968260
4,35185929941418791,0.0002,0.00020,3.0,STAR,STAR,--,Large noise,None,5,...,228,0.839756,2.391115,4.388883,0.654225,1.862836,3.419229,0.918313,0.944213,0.968401
5,35185929941419181,0.4586,0.45860,1.0,GALAXY,GALAXY,--,Poor noise subtraction at 4400. Splice error 5...,None,5,...,53,0.239351,0.855025,1.875840,0.186145,0.664957,1.458849,0.916342,0.942847,0.967617
6,35185929941419381,0.9357,0.93570,4.0,GALAXY,QSO,C,--,None,5,...,211,4.135710,4.465388,7.363845,3.221942,3.478779,5.736834,0.917467,0.943627,0.968065
7,35185929941420031,1.5005,1.50050,1.0,GALAXY,GALAXY,--,--,None,5,...,94,0.483594,0.929053,1.420704,0.376337,0.722996,1.105602,0.918703,0.944483,0.968556
8,35185929941420850,0.0005,0.00050,4.0,STAR,STAR,--,--,None,5,...,248,1.739346,3.288057,4.919703,1.354533,2.560606,3.831267,0.917302,0.943513,0.967999
9,35185929941421006,0.0080,0.00834,2.0,GALAXY,STAR,--,--,Low z no big galaxy in image,5,...,232,0.924085,1.405844,1.309818,0.719449,1.094524,1.019763,0.917224,0.943459,0.967969


In [30]:
(vi['best spectype']=='QSO').sum()

361

In [21]:
## Extract the information we want.
tids = vi['TARGETID']
z_vi = vi['best z']
class_vi = vi['best spectype']
secure = vi['best quality']>=2.5
possible = (vi['best quality']>0) & (~secure)

In [22]:
set(class_vi)

{'GALAXY', 'QSO', 'STAR'}

In [23]:
data = fits.open('/global/homes/j/jfarr/Projects/QuasarNET/data/MiniSV/SV0/1exp/MiniSV_data_1exp_00055626_SV0_QSO_lmax9800.fits')

In [24]:
## Make sure that all of our VI objects are in the data file.
w = np.in1d(tids,data[1].data['TARGETID'])
assert w.sum()==len(tids)

In [25]:
## Construct the dictionary needed for the truth file.
tr_dict = {}
tr_dict['TARGETID'] = tids
tr_dict['Z_VI'] = z_vi

tr_dict['SPID0'] = data[1].data['SPID0'][np.in1d(data[1].data['TARGETID'],tr_dict['TARGETID'])]
tr_dict['SPID1'] = data[1].data['SPID1'][np.in1d(data[1].data['TARGETID'],tr_dict['TARGETID'])]
tr_dict['SPID2'] = data[1].data['SPID2'][np.in1d(data[1].data['TARGETID'],tr_dict['TARGETID'])]

class_vi_coded = np.zeros(class_vi.shape)
class_vi_coded[class_vi=='QSO'] = 3
class_vi_coded[class_vi=='GALAXY'] = 4
class_vi_coded[class_vi=='STAR'] = 1
tr_dict['CLASS_PERSON'] = class_vi_coded

tr_dict['BAL_FLAG_VI'] = np.zeros(tr_dict['TARGETID'].shape)
tr_dict['BI_CIV'] = np.zeros(tr_dict['TARGETID'].shape)

confidences = np.zeros(secure.shape)
confidences[secure] = 3
confidences[possible] = 1
tr_dict['Z_CONF_PERSON'] = confidences.astype('int')

In [26]:
set(tr_dict['CLASS_PERSON'])

{1.0, 3.0, 4.0}

In [27]:
outname = '/global/homes/j/jfarr/Projects/QuasarNET/data/MiniSV/SV0//SV0_truth_tile68002_night20200315.fits'
table = Table(tr_dict)
table.write(outname, format='fits', overwrite=True)

### Make a directory structure with targetids in
For each (plate,mjd) pair, make a directory targetids/{plate}/, and create a file within it targetids-plate-mjd.txt

In [37]:
## Get all spectra for objects in DR12Q supserset
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)

In [39]:
## Get the set of (plate,mjd) pairs that cover these spectra
plate = spall_dr12[1].data['PLATE'][w].astype('i8')
mjd = spall_dr12[1].data['MJD'][w].astype('i8')
fiberid = spall_dr12[1].data['FIBERID'][w].astype('i8')

pm_set = set([(p,m) for p,m in zip(plate,mjd)])
print(len(pm_set))

2493


In [40]:
## Get the list of targetids for these spectra
targetids = platemjdfiber2targetid(plate,mjd,fiberid)

In [46]:
outpath = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/'
for i,(p,m) in enumerate(pm_set):
    w = ((np.array(list(zip(plate,mjd)))==(p,m)).all(axis=1))
    pm_targetids = targetids[w]
    dirname = outpath+'/targetids/{}'.format(p)
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    with open(dirname+'/targetids-{}-{}.txt'.format(p,m),'w') as text_file:
        for t in pm_targetids[:-1]:
            text_file.write('{},'.format(t))
        text_file.write('{}'.format(pm_targetids[-1]))
    print('{:.2%}'.format((i+1)/len(pm_set)),end='\r')

### Combine multiple SQUEzE outputs, removing duplicates in each file.

In [102]:
fi_sq = glob.glob('/project/projectdirs/desi/users/iprafols/SQUEzE/Mini-SV/68002/squeze-bossmodel-*-00055628.fits.gz')
out_filename = '../sq_results/sq_SV0_00055628.fits'
print(len(fi_sq))

10


In [103]:
tables = []
for f_sq in fi_sq:
    sq = fits.open(f_sq)
    w = (~sq[1].data['DUPLICATED'])
    f_table = Table(sq[1].data[w])
    tables.append(f_table)
table = vstack(tables)
table.sort('TARGETID')
table[:5]

TARGETID,TARGET_RA,TARGET_DEC,CMX_TARGET,SPECID,LYB_RATIO,LYB_RATIO_SN,LYB_RATIO2,LYA_RATIO,LYA_RATIO_SN,LYA_RATIO2,SIIV_RATIO,SIIV_RATIO_SN,SIIV_RATIO2,CIV_RATIO,CIV_RATIO_SN,CIV_RATIO2,CIV_BLUE_RATIO,CIV_BLUE_RATIO_SN,CIV_BLUE_RATIO2,CIV_RED_RATIO,CIV_RED_RATIO_SN,CIV_RED_RATIO2,CIII_RATIO,CIII_RATIO_SN,CIII_RATIO2,NEIV_RATIO,NEIV_RATIO_SN,NEIV_RATIO2,MGII_RATIO,MGII_RATIO_SN,MGII_RATIO2,NEV_RATIO,NEV_RATIO_SN,NEV_RATIO2,OII_RATIO,OII_RATIO_SN,OII_RATIO2,HB_RATIO,HB_RATIO_SN,HB_RATIO2,OIII_RATIO,OIII_RATIO_SN,OIII_RATIO2,HA_RATIO,HA_RATIO_SN,HA_RATIO2,Z_TRY,PEAK_SIGNIFICANCE,ASSUMED_LINE,PROB_CLASS1,PROB_CLASS3,PROB_CLASS30,PROB_CLASS35,PROB_CLASS45,PROB_CLASS305,PROB_CLASS4,CLASS_PREDICTED,PROB,DUPLICATED
int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str15,float64,float64,float64,float64,float64,float64,float64,int64,float64,bool
35185923930984962,201.35337359935357,6.822608470758955,9007199254744064,35185923930984962,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.5819250405393256,-1.5830266435216216,0.20766097718733595,0.9407725119872348,-0.576219410651012,0.040108215755704314,0.8806976838225048,-1.032342902905168,0.11105520546150553,0.9479317416965869,-0.6755523535550004,0.04446361985042972,1.0961437322685887,3.409547881567021,0.010333974427315505,0.9902181967604592,-0.12758476985558515,0.010333974427315505,nan,nan,nan,0.8916177052356149,8.26252326317139,hb,0.27184817245193305,0.09727301895671307,0.008371360778606831,0.15744466789172587,0.25017811667940293,0.18032529523585225,0.03455936800576612,1,0.1056443797353199,False
35185923930985294,201.32750079573955,6.8426562871904375,1024,35185923930985294,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,none,nan,nan,nan,nan,nan,nan,nan,-1,nan,False
35185923930985431,201.2808097302484,6.846880974472636,8589935139,35185923930985431,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0072277290067426,0.4947947935190738,0.010775657357436023,1.0156651922920361,1.7423515056361731,0.03258241942392968,1.0404004509364868,2.9797569028753017,0.0011272592199203667,1.0010543698885812,0.09121597220484155,0.014885175056405573,1.0239568940370138,2.661329816464819,0.08520091750333368,0.9633122711325609,-2.279706576688966,0.08520091750333368,nan,nan,nan,0.9297058176706552,11.94845438212471,hb,0.5074075786297046,0.08438045451559383,0.006971029902958535,0.0919263030831775,0.20694747304387293,0.08386778034232445,0.018499380482368478,1,0.09135148441855236,False
35185923930986052,201.2238856442244,6.864334846019875,4096,35185923930986052,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,none,nan,nan,nan,nan,nan,nan,nan,-1,nan,False
35185923935177997,201.5024208514255,6.777666203513355,9007199254742016,35185923935177997,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,none,nan,nan,nan,nan,nan,nan,nan,-1,nan,False


In [104]:
prihdr = fits.Header()
prihdu = fits.PrimaryHDU(header=prihdr)
table = fits.BinTableHDU(table)
hdulist = fits.HDUList([prihdu, table])
hdulist.writeto(out_filename,overwrite=True)
hdulist.close()

### Read the pipeline classes/redshifts from DR12 spall, and put all results into a simplified file.

In [65]:
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)
w.sum()

627751

In [66]:
plate = spall_dr12[1].data['PLATE'][w].astype('i8')
mjd = spall_dr12[1].data['MJD'][w].astype('i8')
fiberid = spall_dr12[1].data['FIBERID'][w].astype('i8')

targetid = platemjdfiber2targetid(plate,mjd,fiberid)
spectype = spall_dr12[1].data['CLASS'][w]
z = spall_dr12[1].data['Z'][w]
zwarn = spall_dr12[1].data['ZWARNING'][w]

thing_id = [pmf2tid_dr12[(p,m,f)] for p,m,f in zip(plate,mjd,fiberid)]

data_list = list(zip(targetid,spectype,z,zwarn,thing_id))
dtype = [('TARGETID','i8'),('SPECTYPE','U8'),('Z','f8'),('ZWARN','i8'),('THING_ID_DR12','i8')]

data = np.array(data_list,dtype=dtype)

In [67]:
table = Table(data)
table[:5]

TARGETID,SPECTYPE,Z,ZWARN,THING_ID_DR12
int64,str8,float64,int64,int64
3586551810016,QSO,2.2423312664031982,0,87897905
3586551810018,QSO,2.259599208831787,0,96906927
3586551810020,QSO,3.1812267303466797,0,96907939
3586551810032,STAR,-0.0031940187327563763,4,87896919
3586551810036,STAR,-0.00046957170707173645,0,87897024


In [68]:
table.write('../dr12pipe_results/dr12pipe_sdr12q.fits',overwrite=True)

### Do the same for DR14

In [82]:
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)
w.sum()

627751

In [83]:
targetid_sdr12q = platemjdfiber2targetid(spall_dr12[1].data['PLATE'][w],spall_dr12[1].data['MJD'][w],spall_dr12[1].data['FIBERID'][w])
targetid_dr14 = platemjdfiber2targetid(spall_dr14[1].data['PLATE'],spall_dr14[1].data['MJD'],spall_dr14[1].data['FIBERID'])
w = np.in1d(targetid_dr14,targetid_sdr12q)

In [85]:
plate = spall_dr14[1].data['PLATE'][w].astype('i8')
mjd = spall_dr14[1].data['MJD'][w].astype('i8')
fiberid = spall_dr14[1].data['FIBERID'][w].astype('i8')

targetid = platemjdfiber2targetid(plate,mjd,fiberid)
spectype = spall_dr14[1].data['CLASS'][w]
z = spall_dr14[1].data['Z'][w]
zwarn = spall_dr14[1].data['ZWARNING'][w]

thing_id = [pmf2tid_dr12[(p,m,f)] for p,m,f in zip(plate,mjd,fiberid)]

data_list = list(zip(targetid,spectype,z,zwarn,thing_id))
dtype = [('TARGETID','i8'),('SPECTYPE','U8'),('Z','f8'),('ZWARN','i8'),('THING_ID_DR12','i8')]

data = np.array(data_list,dtype=dtype)

In [86]:
table.write('../dr14pipe_results/dr14pipe_sdr12q.fits',overwrite=True)